In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv",header = None)
d = data.to_latex()
text_file = open("Output_1.txt", "w")
text_file.write(d)
text_file.close()

In [3]:

train, validate, test = np.split(data, [int(.8*len(data)), int(.9*len(data))])
#print (train)
del train[0]

In [4]:
n_features = 2

user_ratings = train.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))


In [5]:
latent_user_preferences

array([[ 0.427,  0.266],
       [ 0.823,  0.326],
       [ 0.568,  0.467],
       ..., 
       [ 0.716,  0.453],
       [ 0.654,  0.34 ],
       [ 0.02 ,  0.368]])

In [6]:
latent_item_features

array([[ 0.859,  0.691],
       [ 0.236,  0.023],
       [ 0.154,  0.849],
       [ 0.621,  0.575],
       [ 0.302,  0.285],
       [ 0.105,  0.529],
       [ 0.115,  0.418],
       [ 0.329,  0.039],
       [ 0.809,  0.805],
       [ 0.791,  0.629],
       [ 0.204,  0.267],
       [ 0.906,  0.074],
       [ 0.435,  0.059],
       [ 0.341,  0.542],
       [ 0.471,  0.014],
       [ 0.701,  0.767],
       [ 0.559,  0.308],
       [ 0.183,  0.514],
       [ 0.17 ,  0.222],
       [ 0.64 ,  0.494],
       [ 0.306,  0.068],
       [ 0.727,  0.495],
       [ 0.542,  0.764],
       [ 0.723,  0.725],
       [ 0.359,  0.332],
       [ 0.151,  0.234],
       [ 0.887,  0.078],
       [ 0.324,  0.249],
       [ 0.016,  0.058],
       [ 0.475,  0.061],
       [ 0.041,  0.482],
       [ 0.742,  0.755],
       [ 0.504,  0.144],
       [ 0.739,  0.157],
       [ 0.986,  0.498],
       [ 0.485,  0.37 ],
       [ 0.361,  0.747],
       [ 0.841,  0.527],
       [ 0.798,  0.73 ],
       [ 0.91 ,  0.913],


In [7]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating!=99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%1 == 0 ):
            print mse
                    

In [8]:
sgd()

26.0966279631
25.1037342469
24.7772725183
24.5361698065
24.3055392612
24.0742490519
23.8411563128
23.6072645707
23.37418156
23.1436828378


In [9]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 0.881,  0.533,  0.533, ...,  0.981,  0.415,  1.344],
       [ 1.064,  0.654,  0.621, ...,  1.259,  0.503,  1.678],
       [ 1.92 ,  1.152,  1.181, ...,  2.073,  0.904,  2.881],
       ..., 
       [ 1.999,  1.206,  1.217, ...,  2.2  ,  0.942,  3.03 ],
       [ 1.365,  0.827,  0.823, ...,  1.529,  0.644,  2.089],
       [ 0.844,  0.493,  0.548, ...,  0.815,  0.395,  1.195]])

In [10]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]                                                            
comparison_data = pd.DataFrame(values)
del data[0]
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(-7.820|0.881),(8.790|0.533),(-9.660|0.533),(-8.160|-0.509),(-7.520|0.623),(-8.500|1.528),(-9.850|-0.057),(4.170|-0.179),(-8.980|0.012),(-4.760|1.188),...,(2.820|1.639),(99.000|1.187),(99.000|1.949),(99.000|1.110),(99.000|0.987),(99.000|1.321),(-5.630|1.354),(99.000|0.981),(99.000|0.415),(99.000|1.344)
1,(4.080|1.064),(-0.290|0.654),(6.360|0.621),(4.370|-0.585),(-2.380|0.787),(-9.660|1.847),(-0.730|-0.105),(-5.340|-0.190),(8.880|0.011),(9.220|1.481),...,(2.820|2.039),(-4.950|1.440),(-0.290|2.380),(7.860|1.417),(-0.190|1.154),(-2.140|1.562),(3.060|1.533),(0.340|1.259),(-4.320|0.503),(1.070|1.678)
2,(99.000|1.920),(99.000|1.152),(99.000|1.181),(99.000|-1.136),(9.030|1.326),(9.270|3.328),(9.030|-0.093),(9.270|-0.412),(99.000|0.027),(99.000|2.546),...,(99.000|3.519),(99.000|2.582),(99.000|4.225),(9.080|2.352),(99.000|2.185),(99.000|2.907),(99.000|3.039),(99.000|2.073),(99.000|0.904),(99.000|2.881)
3,(99.000|1.347),(8.350|0.822),(99.000|0.800),(99.000|-0.759),(1.800|0.976),(8.160|2.337),(-2.820|-0.111),(6.210|-0.256),(99.000|0.016),(1.840|1.847),...,(99.000|2.545),(99.000|1.819),(99.000|2.997),(0.530|1.747),(99.000|1.484),(99.000|1.998),(99.000|2.003),(99.000|1.549),(99.000|0.636),(99.000|2.091)
4,(8.500|1.699),(4.610|1.001),(-4.170|1.085),(-5.390|-1.060),(1.360|1.110),(1.600|2.943),(7.040|-0.017),(4.610|-0.411),(-0.440|0.029),(5.730|2.168),...,(5.190|3.006),(5.580|2.275),(4.270|3.692),(5.190|1.944),(5.730|2.003),(1.550|2.632),(3.110|2.874),(6.550|1.700),(1.800|0.797),(1.600|2.451)
5,(-6.170|-0.252),(-3.540|-0.164),(0.440|-0.129),(-8.500|0.114),(-7.090|-0.215),(-4.320|-0.439),(-8.690|0.054),(-0.870|0.024),(-6.650|-0.001),(-1.800|-0.390),...,(-3.540|-0.532),(-6.890|-0.346),(-0.680|-0.585),(-2.960|-0.398),(-2.180|-0.242),(-3.350|-0.344),(0.050|-0.281),(-9.080|-0.359),(-5.050|-0.121),(-3.450|-0.442)
6,(99.000|1.172),(99.000|0.689),(99.000|0.752),(99.000|-0.735),(8.590|0.761),(-9.850|2.030),(7.720|-0.007),(8.790|-0.287),(99.000|0.020),(99.000|1.489),...,(99.000|2.065),(99.000|1.569),(99.000|2.543),(99.000|1.330),(99.000|1.387),(2.330|1.820),(99.000|1.997),(99.000|1.163),(99.000|0.550),(99.000|1.683)
7,(6.840|0.533),(3.160|0.297),(9.170|0.375),(-6.210|-0.379),(-8.160|0.293),(-1.700|0.921),(9.270|0.051),(1.410|-0.169),(-5.190|0.013),(-4.420|0.606),...,(7.230|0.849),(-1.120|0.705),(-0.100|1.118),(-5.680|0.491),(-3.160|0.688),(-3.350|0.877),(2.140|1.061),(-0.050|0.417),(1.310|0.247),(0.000|0.684)
8,(-3.790|1.380),(-3.540|0.800),(-9.420|0.907),(-6.890|-0.895),(-8.740|0.861),(-0.290|2.388),(-5.290|0.027),(-8.930|-0.363),(-7.860|0.026),(-1.600|1.706),...,(4.370|2.372),(-0.290|1.841),(4.170|2.969),(-0.290|1.490),(-0.290|1.671),(-0.290|2.176),(-0.290|2.452),(-0.290|1.295),(-3.400|0.645),(-4.950|1.927)
9,(3.010|1.302),(5.150|0.783),(5.150|0.797),(3.010|-0.765),(6.410|0.906),(5.150|2.257),(8.930|-0.070),(2.520|-0.274),(3.010|0.018),(8.160|1.735),...,(99.000|2.397),(4.470|1.752),(99.000|2.869),(99.000|1.609),(99.000|1.474),(99.000|1.965),(99.000|2.043),(99.000|1.419),(99.000|0.613),(99.000|1.963)


In [ ]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.880993387637)","(8.79, 0.532999755714)","(-9.66, 0.532778248334)","(-8.16, -0.509312082135)","(-7.52, 0.622885034311)","(-8.5, 1.52777066438)","(-9.85, -0.0571944718357)","(4.17, -0.178605537874)","(-8.98, 0.011596008887)","(-4.76, 1.18750084793)",...,"(2.82, 1.63878328204)","(99.0, 1.18728778758)","(99.0, 1.94903498405)","(99.0, 1.10997434991)","(99.0, 0.987069593583)","(99.0, 1.32071089717)","(-5.63, 1.35369747434)","(99.0, 0.981376257943)","(99.0, 0.415424065638)","(99.0, 1.34405175736)"
1,"(4.08, 1.0639904023)","(-0.29, 0.654321725113)","(6.36, 0.621199313878)","(4.37, -0.585208820067)","(-2.38, 0.787417772128)","(-9.66, 1.84652821513)","(-0.73, -0.104928700144)","(-5.34, -0.190260749818)","(8.88, 0.0114945725433)","(9.22, 1.48126983473)",...,"(2.82, 2.03891849656)","(-4.95, 1.43965902031)","(-0.29, 2.37969391023)","(7.86, 1.41654770974)","(-0.19, 1.15386599046)","(-2.14, 1.56232404667)","(3.06, 1.53335046148)","(0.34, 1.25943110084)","(-4.32, 0.503370488947)","(1.07, 1.67772902139)"
2,"(99.0, 1.91981451044)","(99.0, 1.15214886414)","(99.0, 1.18057926071)","(99.0, -1.13617521952)","(9.03, 1.32642626716)","(9.27, 3.32799312279)","(9.03, -0.0930828236842)","(9.27, -0.411599255595)","(99.0, 0.0274783798407)","(99.0, 2.54628570598)",...,"(99.0, 3.51858431576)","(99.0, 2.58221348588)","(99.0, 4.22451959096)","(9.08, 2.35190437191)","(99.0, 2.18461740757)","(99.0, 2.90681865307)","(99.0, 3.03925750086)","(99.0, 2.07325997198)","(99.0, 0.903813052675)","(99.0, 2.88092980862)"
3,"(99.0, 1.3469110347)","(8.35, 0.82188853978)","(99.0, 0.799843041148)","(99.0, -0.758912403467)","(1.8, 0.975523443247)","(8.16, 2.33667391441)","(-2.82, -0.111130356852)","(6.21, -0.256251302181)","(99.0, 0.0160702071214)","(1.84, 1.84663835875)",...,"(99.0, 2.54492644602)","(99.0, 1.81899084498)","(99.0, 2.99684523455)","(0.53, 1.74721072061)","(99.0, 1.48382512606)","(99.0, 1.99755722317)","(99.0, 2.00252560771)","(99.0, 1.54940933484)","(99.0, 0.6362171539)","(99.0, 2.09086449539)"
4,"(8.5, 1.6992475002)","(4.61, 1.00051323507)","(-4.17, 1.08534029429)","(-5.39, -1.05992990671)","(1.36, 1.11020422574)","(1.6, 2.94307285757)","(7.04, -0.0172793237146)","(4.61, -0.410516138541)","(-0.44, 0.0288796631642)","(5.73, 2.1682025158)",...,"(5.19, 3.00586328908)","(5.58, 2.27509867398)","(4.27, 3.69229238994)","(5.19, 1.94365810982)","(5.73, 2.00305925346)","(1.55, 2.6322739967)","(3.11, 2.87445655252)","(6.55, 1.70030770849)","(1.8, 0.796967121522)","(1.6, 2.45097945118)"
5,"(-6.17, -0.252217767225)","(-3.54, -0.163764560918)","(0.44, -0.129099198443)","(-8.5, 0.114324316351)","(-7.09, -0.215342644494)","(-4.32, -0.438871598291)","(-8.69, 0.0541346002176)","(-0.87, 0.0243354839833)","(-6.65, -0.000676201250012)","(-1.8, -0.389577229928)",...,"(-3.54, -0.532072740273)","(-6.89, -0.345963768774)","(-0.68, -0.585169585525)","(-2.96, -0.397827423731)","(-2.18, -0.24231857982)","(-3.35, -0.343643147458)","(0.05, -0.280615711618)","(-9.08, -0.359108730835)","(-5.05, -0.120674865092)","(-3.45, -0.442178872505)"
6,"(99.0, 1.17217951296)","(99.0, 0.688715630734)","(99.0, 0.751756170906)","(99.0, -0.735281099268)","(8.59, 0.761003205775)","(-9.85, 2.03000373256)","(7.72, -0.00698171469804)","(8.79, -0.286687623628)","(99.0, 0.0202675514165)","(99.0, 1.48918780517)",...,"(99.0, 2.06528614005)","(99.0, 1.5686225731)","(99.0, 2.54347222266)","(99.0, 1.3303106596)","(99.0, 1.38702155725)","(2.33, 1.82030878028)","(99.0, 1.99684887651)","(99.0, 1.1626908319)","(99.0, 0.549537958506)","(99.0, 1.68323597005)"
7,"(6.84, 0.533064098455)","(3.16, 0.297245936939)","(9.17, 0.375330058032)","(-6.21, -0.37934385188)","(-8.16, 0.293209981067)","(-1.7, 0.92104407521)","(9.27, 0.0507513764115)","(1.41, -0.168644388255)","(-5.19, 0.0129922988893)","(-4.42, 0.606378842348)",...,"(7.23, 0.849374929003)","(-1.12, 0.704700833547)","(-0.1, 1.1178550554)","(-5.68, 0.490648230846)","(-3.16, 0.688273058208)","(-3.35, 0.8770229

In [ ]:
d = comparison_data.to_latex()
text_file = open("comparison_1.txt", "w")
text_file.write(d)
text_file.close()